# Used Cars Price Prediction - Pakwheels

----------

## Import Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

## Load Dataset

In [39]:
df = pd.read_csv('D:/DS Bootcamp/Machine Learning/Used cars Price prediction/Used-Cars-Price-Prediction-Pakwheels/pakwheels_pakistan_automobile_dataset.csv')
df.head()

,title,price,city,model,mileage,fuel_type,transmission,registered,color,assembly,engine_capacity,post_date,price_category,mileage_category,post_day_of_week,vehicle_age
0,Honda N One Premium 2014,2650000,Lahore,2014,82000,Petrol,Automatic,Lahore,Blue,Imported,660,2024-05-04,Medium,Medium,Saturday,10
1,Nissan Note 2020,5400000,Lahore,2020,59000,Hybrid,Automatic,Un-Registered,Silver,Imported,1200,2024-05-04,High,Medium,Saturday,4
2,Suzuki Vitara GLX 1.6 2017,0,Karachi,2017,67000,Petrol,Automatic,Karachi,Grey,Imported,1600,2024-05-04,Low,Medium,Saturday,7
3,Toyota Yaris Cross 2021,7850000,Lahore,2021,41000,Hybrid,Automatic,Un-Registered,Beige,Imported,1500,2024-05-04,High,Low,Saturday,3
4,BMW X1 sDrive18i 2017,10700000,Islamabad,2017,37000,Petrol,Automatic,Islamabad,White,Imported,1500,2024-05-04,High,Low,Saturday,7


In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48189 entries, 0 to 48188
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   title             48189 non-null  object
 1   price             48189 non-null  int64 
 2   city              48189 non-null  object
 3   model             48189 non-null  int64 
 4   mileage           48189 non-null  int64 
 5   fuel_type         48189 non-null  object
 6   transmission      48189 non-null  object
 7   registered        48189 non-null  object
 8   color             48189 non-null  object
 9   assembly          48189 non-null  object
 10  engine_capacity   48189 non-null  int64 
 11  post_date         48189 non-null  object
 12  price_category    48189 non-null  object
 13  mileage_category  48189 non-null  object
 14  post_day_of_week  48189 non-null  object
 15  vehicle_age       48189 non-null  int64 
dtypes: int64(5), object(11)
memory usage: 5.9+ MB


## Data Preprocessing

### Data Inconsisties

- From title extract the Brand name and remove the model year bcz we have already a column called model

In [41]:
df['brand'] = df['title'].str.extract(r'(\w+)', expand=False)
col = df.pop('brand')
df.insert(0, 'brand', col)

In [42]:
df['title'] = df['title'].str.replace(r'\s\d{4}$', '', regex=True)

Rename the column title to model_name and model to model_year

In [43]:
df.rename(columns={'title':'model_name'},inplace=True)
df.rename(columns={'model':'model_year'},inplace=True)

In [44]:
df.columns

Index(['brand', 'model_name', 'price', 'city', 'model_year', 'mileage',
       'fuel_type', 'transmission', 'registered', 'color', 'assembly',
       'engine_capacity', 'post_date', 'price_category', 'mileage_category',
       'post_day_of_week', 'vehicle_age'],
      dtype='object')

Remove the columns price_category, mileage_category, and post_day_of_week because their values are derived from unknown or unclear criteria

In [45]:
df.drop(['price_category', 'mileage_category', 'post_day_of_week'], axis=1, inplace=True)

Correct the dtype of post_date into datetime

In [46]:
df['post_date']=df['post_date'].astype('datetime64[s]')

In [47]:
cat = df.select_dtypes(include=['object']).columns
for i in cat:
    print(df[i].nunique())
    print(df[i].unique())
    print(df[i].value_counts())
    print("_______________________")

63
['Honda' 'Nissan' 'Suzuki' 'Toyota' 'BMW' 'KIA' 'Haval' 'Daihatsu' 'DFSK'
 'MG' 'Mercedes' 'ORA' 'Mitsubishi' 'Hyundai' 'Porsche' 'Lexus' 'Audi'
 'Datsun' 'Adam' 'Sogo' 'Peugeot' 'Changan' 'Chery' 'GMC' 'FAW' 'Jeep'
 'Prince' 'Proton' 'Range' 'Daewoo' 'Isuzu' 'Tesla' 'Mazda' 'BAIC'
 'Chevrolet' 'Ford' 'Subaru' 'Land' 'Volkswagen' 'United' 'Rinco' 'Dodge'
 'Oldsmobile' 'Genesis' 'SsangYong' 'GUGO' 'Fiat' 'JW' 'Cadillac' 'Bad'
 'JMC' 'Jaguar' 'Daehan' 'Renault' 'Power' 'Hino' 'JAC' 'MINI' 'Gateway'
 'ZOTYE' 'Opel' 'Volvo' 'Master']
brand
Toyota        15749
Suzuki        12439
Honda         10257
Daihatsu       2134
Nissan         1276
KIA            1243
Hyundai         982
Mitsubishi      654
Mercedes        495
Changan         484
MG              308
Audi            226
              ...  
JMC               3
Gateway           2
Rinco             2
JAC               2
Volvo             1
Opel              1
ZOTYE             1
Oldsmobile        1
Renault           1
Genesis        

So We have problem in transmission column where around 19109 cars transmission is not avalaible, so we correct those by checking the varient of the car and then assign the transmission type

In [13]:
manual_keywords = ['vx', 'vxr', 'gli 1.3', 'xli', '1.3', 'mt', 'manual','base model', 
                   'euro ii', 'dx', 'exi', 'i-dsi', 'rxi', 'vct-i', 'vxli', 'khyber', 'mehran',
                   'bolan','jxr', 'karvaan', 'sunny', 'cuore', 'margalla', 'jimny','baleno', 'fx']

def manual_cars(car_name):
    car_name = str(car_name).lower()
    return any(keyword in car_name for keyword in manual_keywords)

df['transmission'] = df.apply(lambda row: 'Manual' if row['transmission'] == 'Not Available' and manual_cars(row['model_name']) else row['transmission'],axis=1)

In [48]:
df['transmission'].value_counts()

transmission
Automatic        29079
Not Available    19109
Manual               1
Name: count, dtype: int64

In [51]:
df[df['fuel_type']=='Hybrid']['transmission'].value_counts()

transmission
Automatic        3685
Not Available      15
Name: count, dtype: int64

In [52]:
df[df['fuel_type']=='Petrol']['transmission'].value_counts()

transmission
Automatic        23960
Not Available    17858
Name: count, dtype: int64

In [53]:
df[df['fuel_type']=='Diesel']['transmission'].value_counts()

transmission
Automatic        1136
Not Available     743
Manual              1
Name: count, dtype: int64

In [54]:
df[df['fuel_type'] == 'Diesel']

,brand,model_name,price,city,model_year,mileage,fuel_type,transmission,registered,color,assembly,engine_capacity,post_date,vehicle_age
66,Toyota,Toyota Fortuner GR-S,21300000,Lahore,2023,10000,Diesel,Automatic,Islamabad,White,Local,2800,2024-05-04,1
106,Toyota,Toyota Surf SSR-G 3.0D,2350000,Lahore,2010,50000,Diesel,Automatic,Lahore,Red,Imported,3000,2024-05-04,14
146,Toyota,Toyota Prado TX 2.8 D AT,28500000,Lahore,2018,52000,Diesel,Automatic,Un-Registered,Superior White,Local,2755,2024-05-03,6
161,Toyota,Toyota Fortuner GR-S,21700000,Lahore,2024,10,Diesel,Automatic,Un-Registered,Black,Local,2800,2024-05-03,0
162,Toyota,Toyota Fortuner 2.8 Sigma 4,14000000,Rawalpindi,2020,37000,Diesel,Automatic,Islamabad,Super White,Local,2800,2024-05-03,4
177,Toyota,Toyota Hilux Double Cab,2200000,Peshawar,1992,150000,Diesel,Not Available,Islamabad,White,Local,2400,2024-05-04,32
206,Toyota,Toyota Fortuner Legender,19500000,Lahore,2023,523,Diesel,Automatic,Un-Registered,Super White,Local,2800,2024-05-04,1
222,Adam,Adam Revo,14500000,Rawalpindi,2023,34000,Diesel,Automatic,Islamabad,Black,Local,2900,2024-05-03,1
232,Sogo,Sogo Pickup,1360000,Faisalabad,1993,100000,Diesel,Not Available,Punjab,White,Local,2600,2024-05-04,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [3]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd

In [20]:
url = 'https://www.pakwheels.com/used-cars/search/-/'
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36"
}
response = requests.get(url,headers=headers).text
soup = BeautifulSoup(response, 'html.parser')


In [21]:
soup.find_all('h1')[0].text

'Used Cars For Sale In Pakistan'

In [24]:
# Extract the href attribute
soup.find_all('div', class_='search-title-row')


[<div class="search-title-row">
 <div class="search-title">
 <div class="right">
 <div class="price-details generic-dark-grey">
 
                         PKR 16.5 <span>lacs</span>
 </div>
 </div>
 <a class="car-name ad-detail-path" current-index="0" href="/used-cars/suzuki-mehran-2018-for-sale-in-lahore-9913139" target="_blank" title="Suzuki Mehran  2018 VXR Euro II">
 <h3 style="white-space: normal;">Suzuki Mehran  2018 VXR Euro II for Sale</h3>
 <span class="auction-rating">8.7/10</span>
 </a>
 </div>
 </div>,
 <div class="search-title-row">
 <div class="search-title">
 <div class="right">
 <div class="price-details generic-dark-grey">
 
                         PKR 68.75 <span>lacs</span>
 </div>
 </div>
 <a class="car-name ad-detail-path" current-index="1" href="/used-cars/toyota-raize-2023-for-sale-in-karachi-10060984" target="_blank" title="Toyota Raize  2023 Z">
 <h3 style="white-space: normal;">Toyota Raize  2023 Z for Sale</h3>
 </a>
 </div>
 </div>,
 <div class="search-titl

In [35]:
urls = []
for a_tag in soup.find_all('a', class_='car-name ad-detail-path'):
    href = a_tag.get('href')
    if href:
        urls.append("https://www.pakwheels.com" + href)  # Full URL

# Print extracted URLs
for url in urls:
    print(url)

https://www.pakwheels.com/used-cars/suzuki-mehran-2018-for-sale-in-lahore-9913139
https://www.pakwheels.com/used-cars/toyota-raize-2023-for-sale-in-karachi-10060984
https://www.pakwheels.com/used-cars/toyota-yaris-sedan-2022-for-sale-in-karachi-9433973
https://www.pakwheels.com/used-cars/toyota-aqua-2015-for-sale-in-karachi-9097443
https://www.pakwheels.com/used-cars/honda-civic-2022-for-sale-in-rawalpindi-10036683
https://www.pakwheels.com/used-cars/honda-br-v-2021-for-sale-in-karachi-9665854
https://www.pakwheels.com/used-cars/honda-city-2016-for-sale-in-gujranwala-10060971
https://www.pakwheels.com/used-cars/mazda-carol-2022-for-sale-in-faisalabad-10060975
https://www.pakwheels.com/used-cars/honda-vezel-2016-for-sale-in-islamabad-10000064
https://www.pakwheels.com/used-cars/honda-civic-2023-for-sale-in-rawalpindi-10060892
https://www.pakwheels.com/used-cars/mg-hs-2021-for-sale-in-islamabad-9979281
https://www.pakwheels.com/used-cars/porsche-taycan-2024-for-sale-in-lahore-9881115
htt

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import random

def scrape_car_urls(target_count=40000):
    base_url = "https://www.pakwheels.com/used-cars/search/-/?page="
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36"
    }

    car_urls = set()
    page = 1

    while len(car_urls) < target_count:
        try:
            print(f"Scraping page {page}... (Collected {len(car_urls)} URLs)")
            response = requests.get(base_url + str(page), headers=headers, timeout=10)

            if response.status_code != 200:
                print(f"Warning: Received status code {response.status_code}. Retrying after a short break.")
                time.sleep(5)
                continue

            soup = BeautifulSoup(response.text, 'html.parser')
            listings = soup.find_all('a', class_='car-name ad-detail-path')

            if not listings:
                print("No more listings found. Possibly end of results or blocked.")
                break

            for listing in listings:
                href = listing.get("href")
                if href and href.startswith("/used-cars"):
                    full_url = "https://www.pakwheels.com" + href
                    car_urls.add(full_url)
                    if len(car_urls) >= target_count:
                        break

            page += 1
            time.sleep(random.uniform(1.5, 3.5))  # Respectful delay between requests

        except Exception as e:
            print(f"Error occurred: {e}. Retrying after a short pause.")
            time.sleep(5)

    return list(car_urls)

# Run the scraper and store results
car_links = scrape_car_urls(target_count=40000)

# Summary
print(f"\n✅ Successfully collected {len(car_links)} unique car URLs.")


Scraping page 1... (Collected 0 URLs)
Scraping page 2... (Collected 29 URLs)
Scraping page 3... (Collected 56 URLs)
Scraping page 4... (Collected 91 URLs)
Scraping page 5... (Collected 124 URLs)
Scraping page 6... (Collected 158 URLs)
Scraping page 7... (Collected 186 URLs)
Scraping page 8... (Collected 217 URLs)
Scraping page 9... (Collected 244 URLs)
Scraping page 10... (Collected 272 URLs)
Scraping page 11... (Collected 302 URLs)
Scraping page 12... (Collected 333 URLs)
Scraping page 13... (Collected 357 URLs)
Scraping page 14... (Collected 387 URLs)
Scraping page 15... (Collected 417 URLs)
Scraping page 16... (Collected 449 URLs)
Scraping page 17... (Collected 480 URLs)
Scraping page 18... (Collected 508 URLs)
Scraping page 19... (Collected 538 URLs)
Scraping page 20... (Collected 563 URLs)
Scraping page 21... (Collected 594 URLs)
Scraping page 22... (Collected 626 URLs)
Scraping page 23... (Collected 657 URLs)
Scraping page 24... (Collected 685 URLs)
Scraping page 25... (Collected

In [8]:
df = pd.DataFrame({'car_links': car_links})
df.to_csv('car_links.csv', index=False)

In [7]:
car_links = pd.read_csv('D:/DS Bootcamp/Machine Learning/Used cars Price prediction/Used-Cars-Price-Prediction-Pakwheels/car_links.csv')
car_links.head()

,car_links
0,https://www.pakwheels.com/used-cars/suzuki-meh...
1,https://www.pakwheels.com/used-cars/mitsubishi...
2,https://www.pakwheels.com/used-cars/suzuki-wag...
3,https://www.pakwheels.com/used-cars/toyota-lan...
4,https://www.pakwheels.com/used-cars/suzuki-alt...


In [20]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random

# Slice first 50 links
car_links_50 = car_links.head(5)

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

data = {
    "title": [],
    "location": [],
    "model_year": [],
    "fuel_type": [],
    "transmission": [],
    "km_driven": [],
    "registered_in": [],
    "color": [],
    "assembly": [],
    "engine_capacity": [],
    "body_type": [],
    "last_updated": [],
    "price": [],
    "url": []
}

for i, row in car_links_50.iterrows():
    url = row['car_links']
    print(f"Scraping {i+1}/50: {url}")
    try:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")

        # Title and location
        title = soup.find("h1")
        location_tag = soup.find("p", class_="detail-sub-heading").find("a", href="#")
        
        data["title"].append(title.text.strip() if title else "N/A")
        data["location"].append(location_tag.text.strip() if location_tag else "N/A")

        # Model Year
        model_year_tag = soup.find("a", title=lambda x: x and x.startswith("Year"))
        data["model_year"].append(model_year_tag.text.strip() if model_year_tag else "N/A")

        # Fuel Type
        fuel_type_tag = soup.find("span", class_="engine-icon type")
        fuel_type = fuel_type_tag.find_next("p").text.strip() if fuel_type_tag else "N/A"
        data["fuel_type"].append(fuel_type)

        # Transmission Type
        transmission_tag = soup.find("span", class_="engine-icon transmission")
        transmission = transmission_tag.find_next("p").text.strip() if transmission_tag else "N/A"
        data["transmission"].append(transmission)

        # KM Driven
        km_driven_tag = soup.find("span", class_="engine-icon millage")
        km_driven = km_driven_tag.find_next("p").text.strip() if km_driven_tag else "N/A"
        data["km_driven"].append(km_driven)

        # Specs (Registered In, Color, Assembly, Engine Capacity, Body Type, Last Updated)
        specs = soup.find_all("li", class_="ad-data")
        if len(specs) > 0:
            data["registered_in"].append(specs[1].find_next("li").text.strip() if len(specs) > 1 else "N/A")
            data["color"].append(specs[3].find_next("li").text.strip() if len(specs) > 3 else "N/A")
            data["assembly"].append(specs[5].find_next("li").text.strip() if len(specs) > 5 else "N/A")
            data["engine_capacity"].append(specs[7].find_next("li").text.strip() if len(specs) > 7 else "N/A")
            data["body_type"].append(specs[9].find_next("li").text.strip() if len(specs) > 9 else "N/A")
            data["last_updated"].append(specs[11].find_next("li").text.strip() if len(specs) > 11 else "N/A")

        # Price
        price_tag = soup.find("strong", class_="generic-green")
        if price_tag:
            price = price_tag.text.strip()
            price = price.replace("lacs", "Lakh")  # Optional: replace "lacs" with "Lakh"
            data["price"].append(price)
        else:
            data["price"].append("N/A")

        # URL
        data["url"].append(url)

    except Exception as e:
        print(f"Error scraping {url}: {e}")
        for key in data:
            if key != "url":
                data[key].append("N/A")
        data["url"].append(url)

    # Adding a random delay to avoid getting blocked
    time.sleep(random.uniform(1.5, 3.5))

# Save to CSV
df_50 = pd.DataFrame(data)
df_50.to_csv("pakwheels_50_cars.csv", index=False)
print("Scraping complete. Saved to 'pakwheels_50_cars.csv'.")


Scraping 1/50: https://www.pakwheels.com/used-cars/suzuki-mehran-2003-for-sale-in-multan-10059331
Scraping 2/50: https://www.pakwheels.com/used-cars/mitsubishi-lancer-2006-for-sale-in-lahore-10025412
Scraping 3/50: https://www.pakwheels.com/used-cars/suzuki-wagon-r-2013-for-sale-in-lahore-10027103
Scraping 4/50: https://www.pakwheels.com/used-cars/toyota-land-cruiser-2019-for-sale-in-lahore-10025538
Scraping 5/50: https://www.pakwheels.com/used-cars/suzuki-alto-2021-for-sale-in-faisalabad-10034114
Scraping complete. Saved to 'pakwheels_50_cars.csv'.


In [27]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random

car_links = car_links.head(10000)

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

# Initialize a dictionary to store car data
data = {
    "title": [],
    "location": [],
    "model_year": [],
    "fuel_type": [],
    "transmission": [],
    "km_driven": [],
    "registered_in": [],
    "color": [],
    "assembly": [],
    "engine_capacity": [],
    "body_type": [],
    "last_updated": [],
    "price": [],
    "url": []
}

# Loop through each car link to scrape details
for i, row in car_links.iterrows():
    url = row['car_links']
    print(f"Scraping {i+1}: {url}")
    try:
        # Make a request to the car ad page
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")

        # Title and Location
        title = soup.find("h1")
        location_tag = soup.find("p", class_="detail-sub-heading").find("a", href="#")
        data["title"].append(title.text.strip() if title else "N/A")
        data["location"].append(location_tag.text.strip() if location_tag else "N/A")

        # Model Year
        model_year_tag = soup.find("a", title=lambda x: x and x.startswith("Year"))
        data["model_year"].append(model_year_tag.text.strip() if model_year_tag else "N/A")

        # Fuel Type
        fuel_type_tag = soup.find("span", class_="engine-icon type")
        fuel_type = fuel_type_tag.find_next("p").text.strip() if fuel_type_tag else "N/A"
        data["fuel_type"].append(fuel_type)

        # Transmission Type
        transmission_tag = soup.find("span", class_="engine-icon transmission")
        transmission = transmission_tag.find_next("p").text.strip() if transmission_tag else "N/A"
        data["transmission"].append(transmission)

        # KM Driven
        km_driven_tag = soup.find("span", class_="engine-icon millage")
        km_driven = km_driven_tag.find_next("p").text.strip() if km_driven_tag else "N/A"
        data["km_driven"].append(km_driven)

        # Specs (Registered In, Color, Assembly, Engine Capacity, Body Type, Last Updated)
        specs = soup.find_all("li", class_="ad-data")
        # Use list indexing to handle each specification
        data["registered_in"].append(specs[0].find_next("li").text.strip() if len(specs) > 0 else "N/A")
        data["color"].append(specs[1].find_next("li").text.strip() if len(specs) > 1 else "N/A")
        data["assembly"].append(specs[2].find_next("li").text.strip() if len(specs) > 2 else "N/A")
        data["engine_capacity"].append(specs[3].find_next("li").text.strip() if len(specs) > 3 else "N/A")
        data["body_type"].append(specs[4].find_next("li").text.strip() if len(specs) > 4 else "N/A")
        data["last_updated"].append(specs[5].find_next("li").text.strip() if len(specs) > 5 else "N/A")

        # Price (optional handling of "lacs" to "Lakh")
        price_tag = soup.find("strong", class_="generic-green")
        if price_tag:
            price = price_tag.text.strip()
            price = price.replace("lacs", "Lakh")  # Optional: replace "lacs" with "Lakh"
            data["price"].append(price)
        else:
            data["price"].append("N/A")

        # URL (to be added in the data)
        data["url"].append(url)

    except Exception as e:
        # If an error occurs, log it and append "N/A" for missing data
        print(f"Error scraping {url}: {e}")
        for key in data:
            if key != "url":
                data[key].append("N/A")
        data["url"].append(url)

    # Adding a random delay to avoid being blocked
    time.sleep(random.uniform(1.2, 3))

# Save the scraped data to a CSV file
df = pd.DataFrame(data)
df.to_csv("Pakwheels_used_cars.csv", index=False)


Scraping 1: https://www.pakwheels.com/used-cars/suzuki-mehran-2003-for-sale-in-multan-10059331
Scraping 2: https://www.pakwheels.com/used-cars/mitsubishi-lancer-2006-for-sale-in-lahore-10025412
Scraping 3: https://www.pakwheels.com/used-cars/suzuki-wagon-r-2013-for-sale-in-lahore-10027103
Scraping 4: https://www.pakwheels.com/used-cars/toyota-land-cruiser-2019-for-sale-in-lahore-10025538
Scraping 5: https://www.pakwheels.com/used-cars/suzuki-alto-2021-for-sale-in-faisalabad-10034114
Scraping 6: https://www.pakwheels.com/used-cars/mg-zs-2021-for-sale-in-lahore-10059170
Scraping 7: https://www.pakwheels.com/used-cars/suzuki-wagon-r-2018-for-sale-in-islamabad-10039539
Scraping 8: https://www.pakwheels.com/used-cars/toyota-fortuner-2023-for-sale-in-vehari-10061008
Scraping 9: https://www.pakwheels.com/used-cars/suzuki-wagon-r-2018-for-sale-in-gujrat-10005331
Scraping 10: https://www.pakwheels.com/used-cars/toyota-prado-2016-for-sale-in-sialkot-9888238
Scraping 11: https://www.pakwheels.com